<a href="https://colab.research.google.com/github/jayshah1819/tensorrt-int8-practice/blob/main/INT8_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!dpkg -l | grep nvinfer


In [1]:
!pip install torch torchvision ultralytics
!apt-get update
!apt-get install -y tensorrt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.7 MB/s eta 0:00:00
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,798 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,638 kB]
Hit:13 https://ppa.l

In [2]:
!pip install tensorrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.9 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.13.3.9-py2.py3-none-any.whl size=46401 sha256=7008b02352fd65fbdecfdb3552a1f871ea4a731443b592dbe5e8f0b55d27aca0
  Stored in directory: /root/.cache/pip/wheels/1a/aa/b0/561559321c1659e43c5a6661986c3072f9c383efeeaaffb1a5
  Created wheel for tensorrt_cu13: filename=tensorrt_cu13-10.13.3.9-py2.py3-none-any.whl size=17436 sha256=f61ea41e009ae4c11713094602c372c15aa3c58adc9814b5d985d5933d33f583
  Stored in directory: /root/.cache/pip/wheels/df/4a/b2/ebfc5437a397c53faff0dca4a36c096d4b7d2b7436a2707e60
  Created wheel for tensorrt_cu13_libs: filename=tensorrt_cu13_libs-10.13.3.9-

Step 1) Defining the network

Step 2) Setting up the INT8 calibrator

Step 3) Configuring the builder

Step 4) Building the TensorRT engine

Step 5) Running inference

Step 6) Verifying the output **bold text**

In [3]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 10.7 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.2-cp312-cp312-linux_x86_64.whl size=659050 sha256=5bfa38fd8580336fa63c80c5b848bddf64db80fe5748c0c2d5bfe326cc5a1e1c
  Stored in directory: /root/.cache/pip/wheels/d5/36/f3/ac5f09d768cad3fa15d5a3449bdfe65c3de58e69d036c73228
Successfully built pycuda


In [4]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np


In [10]:
!pip install ultralytics --quiet
!pip install torch torchvision --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.2 MB/s eta 0:00:00


In [11]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [12]:

onnx_path = "/content/yolov8n.pt"
model.export(format="onnx", opset=17, dynamic=False, simplify=True, imgsz=640)



Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)
requirements: Ultralytics requirements ['onnxslim>=0.1.67', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 6.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.0 opset 17...
ONNX: slimming with onnxslim 0.1.68...
ONNX: export success ✅ 10.3s, saved as 'yolov8n.onnx' (12.2 MB)

Export complete (10.9s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8

'yolov8n.onnx'

In [6]:
import cv2
import numpy as np

video_path = "/content/4K Road traffic video for object detection and tracking - free download now! - Karol Majek (360p, h264).mp4"


cap = cv2.VideoCapture(video_path)
calibration_frames = []
frame_count = 0
max_frames = 100

while cap.isOpened() and frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 640))
    frame = frame.astype(np.float32) / 255.0
    frame = np.transpose(frame, (2, 0, 1))
    calibration_frames.append(frame)
    frame_count += 1

cap.release()

# Stack frames for easier batching
calibration_data = np.array(calibration_frames)
print("Calibration data shape:", calibration_data.shape)


Calibration data shape: (100, 3, 640, 640)


 Defining the network

1.   Prepare Representative Dataset
2.   Create INT8 Calibrator





In [ ]:
calibration_images=np.random.rand(32,3,224,224).astype(np.float32)

In [7]:
!nvidia-smi


Wed Sep 17 19:48:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P0             29W /   70W |     102MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import pycuda.driver as cuda
import tensorrt as trt

class MyCalibrator(trt.IInt8EntropyCalibrator2):
    def __init__(self, data, batch_size=8, gpu_id=0):
        super().__init__()

        # Manual CUDA init
        cuda.init()
        self.device = cuda.Device(gpu_id)
        self.ctx = self.device.make_context()

        self.data = data
        self.batch_size = batch_size
        self.index = 0
        self.device_input = None

    def get_batch_size(self):
        return self.batch_size

    def get_batch(self, names):
        if self.index + self.batch_size > len(self.data):
            return None
        batch = self.data[self.index:self.index+self.batch_size].ravel()
        if self.device_input is None:
            self.device_input = cuda.mem_alloc(batch.nbytes)
        cuda.memcpy_htod(self.device_input, batch)
        self.index += self.batch_size
        return [int(self.device_input)]

    def read_calibration_cache(self):
        try:
            with open("calib_cache.bin", "rb") as f:
                return f.read()
        except FileNotFoundError:
            return None

    def write_calibration_cache(self, cache):
        with open("calib_cache.bin", "wb") as f:
            f.write(cache)

    def __del__(self):
        if hasattr(self, 'ctx'):
            self.ctx.pop()
            self.ctx.detach()


In [ ]:


TRT_LOGGER = trt.Logger(trt.Logger.INFO)
builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(flags=1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
parser = trt.OnnxParser(network, TRT_LOGGER)

with open(onnx_path, 'rb') as f:
    parser.parse(f.read())
